In [ ]:
import socket
import logging
import ssl
import re
import time
import threading

from tqdm import tqdm 

import dns.message
import dns.query
import dns.exception
import dns.rcode
import dns.name
import pandas as pd

logging.basicConfig(level=logging.WARNING)

In [ ]:
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

In [ ]:
NS = [m[0] for m in re.finditer(r'[^\s\"]+\.[ac]\.desec\.io', "\n".join(list(open('hosts/all.yml', 'r'))))] + ['ns1.desec.io', 'ns2.desec.org']

In [ ]:
NS_NET = {
    'ams-1.a.desec.io': 'a1',
    'dfw-1.a.desec.io': 'a1',
    'fra-1.a.desec.io': 'a2',
    'hkg-1.a.desec.io': 'a2',
    'jnb-1.a.desec.io': 'a2',
    'sao-1.a.desec.io': 'a1',
    'syd-1.a.desec.io': 'a2',
    'dxb-1.c.desec.io': 'c1',
    'fra-1.c.desec.io': 'c1',
    'lax-1.c.desec.io': 'c1',
    'lga-1.c.desec.io': 'c2',
    'lhr-1.c.desec.io': 'c2',
    'scl-1.c.desec.io': 'c2',
    'sin-1.c.desec.io': 'c1',
    'tyo-1.c.desec.io': 'c2',
    'ns1.desec.io': 'ns1',
    'ns2.desec.org': 'ns2',
}

In [ ]:
assert set(NS) == set(NS_NET.keys())

In [ ]:
def run():
    for ns in tqdm(NS):
        for qname in ['external-timestamp.desec.test']:
            for qtype in ['TXT']:
                for addr in {sockaddr[0] for (_, _, _, _, sockaddr) in socket.getaddrinfo(ns, 53)}:
                    if ('.a.' in ns or '.c.' in ns) and ':' in addr:
                        logging.debug(f"Skipping v6 addr for {ns} ({addr})")
                        continue
                    for query in [dns.query.udp, dns.query.tcp, dns.query.tls, dns.query.quic]:
                        logging.debug(f"Query {qname}/{qtype} at {ns}/{addr} using {query.__name__}")
                        q = dns.message.make_query(qname, qtype)
                        match query:
                            case dns.query.quic:
                                kwargs = dict(verify=False, timeout=3)
                            case dns.query.tls:
                                kwargs = dict(ssl_context=ssl_context, timeout=3)
                            case _:
                                kwargs = dict(timeout=1)
                                
                        # TODO replace this madness with a regular try/except block 
                        # once dns.query.quic properly throws exceptions 
                        # if no server is present
                        responses = []
                        def do_query():
                            responses.append(query(q, where=addr, **kwargs))
                        t = threading.Thread(target=do_query)
                        t.start()
                        t.join(timeout=3)

                        yield {
                            'query': q,
                            'response': responses[0] if responses else None,
                            'ns': ns,
                            'addr': addr,
                            'transport': query.__name__,
                            'timestamp': int(time.time()),
                        }

raw_data = list(run())

In [ ]:
data = pd.DataFrame(raw_data)
data['qname'] = data['query'].apply(lambda q: q.question[0].name.to_text())
data['qtype'] = data['query'].apply(lambda q: q.question[0].rdtype)
data['ip'] = data['addr'].apply(lambda addr: 4 if '.' in addr else 6)
data['rcode'] = data['response'].apply(lambda r: dns.rcode.to_text(r.rcode()) if r else None)
data['response_data'] = data['response'].apply(lambda r: int(r.answer[0][0].to_text().strip("\"")) if r else None)
data['ns_net'] = data['ns'].apply(lambda ns: NS_NET[ns])
data['lag'] = data['response_data'] - data['timestamp']

In [ ]:
data.sample(4)

In [ ]:
def highlight_rcode(v):
    if v == 'None':
        color = 'grey'
    elif v == 'NOERROR':
        color = 'green'
    else:
        color = 'yellow'
        
    return f'background-color: {color}'

data.sort_values(['ns_net', 'ns']).pivot(index=('ns_net', 'ns', 'addr'), columns='transport', values='rcode').style.applymap(highlight_rcode)

In [ ]:
def highlight_timestamp(v):
    if pd.isna(v):
        color = 'grey'
    elif v < -600:
        color = 'red'
    elif v < -300:
        color = 'yellow'
    else:
        color = 'green'
        
    return f'background-color: {color}'
    
freshness = data.sort_values(['ns_net', 'ns']).pivot(index=('ns_net', 'ns', 'addr'), columns='transport', values='lag')
freshness.style.applymap(highlight_timestamp).format(lambda v: "---" if pd.isna(v) else f"{v:.0f}s")